In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.by import By


from bs4 import BeautifulSoup
import time

In [10]:
chrome_options = Options()
chrome_options.add_argument("--headless") 
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [11]:
url = 'https://ge.globo.com/futebol/brasileirao-serie-a/'
driver.get(url)
# Pegar o HTML renderizado
html = driver.page_source

# Fechar o navegador após capturar o HTML
driver.quit()

In [15]:
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")
for i, section in enumerate(sections):
    print(f"Seção {i+1}:")
    print(section.get_text(strip=True))  
    print("-" * 40)  # 

In [ ]:
sections = soup.find_all('section',class_="tabela tabela__pontos-corridos")
for i, section in enumerate(sections):
    tables = section.find_all('table', class_="tabela__equipes tabela__equipes--com-borda")  # Encontrar todas as tabelas dentro da seção
    if tables:
        print(f"Seção {i+1} contém {len(tables)} tabela(s):")
        for j, table in enumerate(tables):
            print(f"Tabela {j+1}:")
            rows = table.find_all('tr')  # Encontrar todas as linhas na tabela

            for row in rows:
                cols = row.find_all(['td', 'th'])  # Acessar células da tabela
                cols_text = [col.get_text(strip=True) for col in cols]  # Extrair texto das células
                print(cols_text)  # Mostrar o conteúdo da linha
            print("-" * 40)  # Separador entre tabelas

In [ ]:
sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")
for i, section in enumerate(sections):
    tables = section.find_all('table',class_="tabela__pontos")  
    if tables:
        print(f"Seção {i+1} contém {len(tables)} tabela(s):")
        for j, table in enumerate(tables):
            print(f"Tabela {j+1}:")
            rows = table.find_all('tr')  

            for row in rows:
                cols = row.find_all(['td', 'th'])  
                cols_text = [col.get_text(strip=True) for col in cols]  
                print(cols_text)  
            print("-" * 40)  

In [ ]:
import re

sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")

# Para armazenar as tabelas combinadas
tabela_completa = []

for section in sections:
    # 1. Encontrar a tabela de equipes (nomes, posições, etc.)
    tabela_equipes = section.find_all('table', class_="tabela__equipes tabela__equipes--com-borda")
    
    # 2. Encontrar a tabela de pontos (estatísticas: jogos, vitórias, derrotas, etc.)
    tabela_pontos = section.find_all('table', class_="tabela__pontos")
    
    if tabela_equipes and tabela_pontos:  # Se ambas as tabelas existirem
        # 3. Iterar sobre as tabelas de equipes e pontos em paralelo
        for table_equipes, table_pontos in zip(tabela_equipes, tabela_pontos):
            rows_equipes = table_equipes.find_all('tr')
            rows_pontos = table_pontos.find_all('tr')
            
            # Combinar as informações linha a linha
            for row_equipe, row_ponto in zip(rows_equipes, rows_pontos):
                # Extrair as informações das equipes (nome do time, posição)
                cols_equipe = row_equipe.find_all(['td', 'th'])
                cols_text_equipe = [col.get_text(strip=True) for col in cols_equipe]

                # Remover sufixos dos nomes dos times
                if len(cols_text_equipe) > 1:
                    nome_time = cols_text_equipe[1].replace('BOT', '').replace('PAL', '').replace('FOR', '').replace('FLA', '').replace('SAO', '').replace('INT', '').replace('BAH', '').replace('CRU', '').replace('CAM', '').replace('VAS', '').replace('GRE', '').replace('CRI', '').replace('RBB', '').replace('JUV', '').replace('CAP', '').replace('FLU', '').replace('VIT', '').replace('COR', '').replace('CUI', '').replace('ACG', '').strip()
                    cols_text_equipe[1] = nome_time

                # Extrair as informações dos pontos (estatísticas)
                cols_ponto = row_ponto.find_all(['td', 'th'])
                cols_text_ponto = [col.get_text(strip=True) for col in cols_ponto]

                # 4. Procurar a coluna de "últimos jogos"
                ultimos_jogos_col = row_ponto.find_all('td', class_="classificacao__pontos classificacao__pontos--ultimos_jogos")
                ultimos_jogos_resultados = []
                
                for col in ultimos_jogos_col:
                    spans = col.find_all('span')
                    for span in spans:
                        span_class = span.get('class', [])
                        
                        # Mapeando vitórias, empates e derrotas
                        if "classificacao__ultimos_jogos--v" in span_class:
                            ultimos_jogos_resultados.append('V')  # Vitória
                        elif "classificacao__ultimos_jogos--e" in span_class:
                            ultimos_jogos_resultados.append('E')  # Empate
                        elif "classificacao__ultimos_jogos--d" in span_class:
                            ultimos_jogos_resultados.append('D')  # Derrota

                # 5. Combinar todas as informações na mesma linha (equipe + pontos)
                linha_completa = cols_text_equipe + cols_text_ponto
                
                # Verifica se a última coluna é vazia e a remove, caso seja
                if linha_completa[-1] == '':
                    linha_completa.pop()  # Remove o último elemento se for vazio

                # Insira os resultados dos últimos jogos na posição correta
                if ultimos_jogos_resultados:  # Apenas adiciona se houver resultados
                    linha_completa.append(' '.join(ultimos_jogos_resultados))  # Adiciona como string
                else:
                    linha_completa.append('')  # Adiciona string vazia se não houver resultados

                # Verifica se a linha começa com "Classificação" e ignora
                if linha_completa[0] != "Classificação":  # Ignora a linha de título
                    tabela_completa.append(linha_completa)

print("Tabela Completa:")
for linha in tabela_completa:
    if len(linha) > 3:  # Verifica se a linha tem mais de 3 colunas
        linha.pop(2)  # Remove o valor na posição 3 (index 2)
    print(linha)


In [ ]:
import re

sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")

# Para armazenar as tabelas combinadas
tabela_completa = []

# Defina o cabeçalho da tabela
cabecalho = ['Posição','Time', 'P', 'J', 'V', 'E', 'D', 'GP', 'GC', 'SG', '%', 'ÚLT. JOGOS']

for section in sections:
    # 1. Encontrar a tabela de equipes (nomes, posições, etc.)
    tabela_equipes = section.find_all('table', class_="tabela__equipes tabela__equipes--com-borda")
    
    # 2. Encontrar a tabela de pontos (estatísticas: jogos, vitórias, derrotas, etc.)
    tabela_pontos = section.find_all('table', class_="tabela__pontos")
    
    if tabela_equipes and tabela_pontos:  # Se ambas as tabelas existirem
        # 3. Iterar sobre as tabelas de equipes e pontos em paralelo
        for table_equipes, table_pontos in zip(tabela_equipes, tabela_pontos):
            rows_equipes = table_equipes.find_all('tr')
            rows_pontos = table_pontos.find_all('tr')
            
            # Combinar as informações linha a linha
            for row_equipe, row_ponto in zip(rows_equipes, rows_pontos):
                # Extrair as informações das equipes (nome do time, posição)
                cols_equipe = row_equipe.find_all(['td', 'th'])
                cols_text_equipe = [col.get_text(strip=True) for col in cols_equipe]

                # Remover sufixos dos nomes dos times
                if len(cols_text_equipe) > 1:
                    nome_time = cols_text_equipe[1].replace('BOT', '').replace('PAL', '').replace('FOR', '').replace('FLA', '').replace('SAO', '').replace('INT', '').replace('BAH', '').replace('CRU', '').replace('CAM', '').replace('VAS', '').replace('GRE', '').replace('CRI', '').replace('RBB', '').replace('JUV', '').replace('CAP', '').replace('FLU', '').replace('VIT', '').replace('COR', '').replace('CUI', '').replace('ACG', '').strip()
                    cols_text_equipe[1] = nome_time

                # Extrair as informações dos pontos (estatísticas)
                cols_ponto = row_ponto.find_all(['td', 'th'])
                cols_text_ponto = [col.get_text(strip=True) for col in cols_ponto]

                # 4. Procurar a coluna de "últimos jogos"
                ultimos_jogos_col = row_ponto.find_all('td', class_="classificacao__pontos classificacao__pontos--ultimos_jogos")
                ultimos_jogos_resultados = []
                
                for col in ultimos_jogos_col:
                    spans = col.find_all('span')
                    for span in spans:
                        span_class = span.get('class', [])
                        
                        # Mapeando vitórias, empates e derrotas
                        if "classificacao__ultimos_jogos--v" in span_class:
                            ultimos_jogos_resultados.append('V')  # Vitória
                        elif "classificacao__ultimos_jogos--e" in span_class:
                            ultimos_jogos_resultados.append('E')  # Empate
                        elif "classificacao__ultimos_jogos--d" in span_class:
                            ultimos_jogos_resultados.append('D')  # Derrota

                # 5. Combinar todas as informações na mesma linha (equipe + pontos)
                linha_completa = cols_text_equipe + cols_text_ponto
                
                # Verifica se a última coluna é vazia e a remove, caso seja
                if linha_completa[-1] == '':
                    linha_completa.pop()  # Remove o último elemento se for vazio

                # Insira os resultados dos últimos jogos na posição correta
                if ultimos_jogos_resultados:  # Apenas adiciona se houver resultados
                    linha_completa.append(' '.join(ultimos_jogos_resultados))  # Adiciona como string
                else:
                    linha_completa.append('')  # Adiciona string vazia se não houver resultados

                # Verifica se a linha começa com "Classificação" e ignora
                if linha_completa[0] != "Classificação":  # Ignora a linha de título
                    tabela_completa.append(linha_completa)

# Adicione o cabeçalho à tabela
tabela_completa.insert(0, cabecalho)

# Imprimir a tabela completa
print("Tabela Completa:")
for linha in tabela_completa:
    if len(linha) > 3:  # Verifica se a linha tem mais de 3 colunas
        linha.pop(2)  # Remove o valor na posição 3 (index 2)
    print(linha)


In [ ]:
import re
from tabulate import tabulate

sections = soup.find_all('section', class_="tabela tabela__pontos-corridos")

# Para armazenar as tabelas combinadas
tabela_completa = []

# Adicionar o cabeçalho à tabela
cabecalho = ['Posição', 'Time', 'P', 'J', 'V', 'E', 'D', 'GP', 'GC', 'SG', '%', 'ÚLT. JOGOS']
tabela_completa.append(cabecalho)

for section in sections:
    # 1. Encontrar a tabela de equipes (nomes, posições, etc.)
    tabela_equipes = section.find_all('table', class_="tabela__equipes tabela__equipes--com-borda")
    
    # 2. Encontrar a tabela de pontos (estatísticas: jogos, vitórias, derrotas, etc.)
    tabela_pontos = section.find_all('table', class_="tabela__pontos")
    
    if tabela_equipes and tabela_pontos:  # Se ambas as tabelas existirem
        # 3. Iterar sobre as tabelas de equipes e pontos em paralelo
        for table_equipes, table_pontos in zip(tabela_equipes, tabela_pontos):
            rows_equipes = table_equipes.find_all('tr')
            rows_pontos = table_pontos.find_all('tr')
            
             # Combinar as informações linha a linha
            for row_equipe, row_ponto in zip(rows_equipes, rows_pontos):
                # Extrair as informações das equipes (nome do time, posição)
                cols_equipe = row_equipe.find_all(['td', 'th'])
                cols_text_equipe = [col.get_text(strip=True) for col in cols_equipe]

                # Remover sufixos dos nomes dos times
                if len(cols_text_equipe) > 1:
                    nome_time = cols_text_equipe[1].replace('BOT', '').replace('PAL', '').replace('FOR', '').replace('FLA', '').replace('SAO', '').replace('INT', '').replace('BAH', '').replace('CRU', '').replace('CAM', '').replace('VAS', '').replace('GRE', '').replace('CRI', '').replace('RBB', '').replace('JUV', '').replace('CAP', '').replace('FLU', '').replace('VIT', '').replace('COR', '').replace('CUI', '').replace('ACG', '').strip()
                    cols_text_equipe[1] = nome_time

                # Extrair as informações dos pontos (estatísticas)
                cols_ponto = row_ponto.find_all(['td', 'th'])
                cols_text_ponto = [col.get_text(strip=True) for col in cols_ponto]

                # 4. Procurar a coluna de "últimos jogos"
                ultimos_jogos_col = row_ponto.find_all('td', class_="classificacao__pontos classificacao__pontos--ultimos_jogos")
                ultimos_jogos_resultados = []
                
                for col in ultimos_jogos_col:
                    spans = col.find_all('span')
                    for span in spans:
                        span_class = span.get('class', [])
                        
                        # Mapeando vitórias, empates e derrotas
                        if "classificacao__ultimos_jogos--v" in span_class:
                            ultimos_jogos_resultados.append('V')  # Vitória
                        elif "classificacao__ultimos_jogos--e" in span_class:
                            ultimos_jogos_resultados.append('E')  # Empate
                        elif "classificacao__ultimos_jogos--d" in span_class:
                            ultimos_jogos_resultados.append('D')  # Derrota

                # 5. Combinar todas as informações na mesma linha (equipe + pontos)
                linha_completa = cols_text_equipe + cols_text_ponto
                
                # Verifica se a última coluna é vazia e a remove, caso seja
                if linha_completa[-1] == '':
                    linha_completa.pop()  # Remove o último elemento se for vazio

                # Insira os resultados dos últimos jogos na posição correta
                if ultimos_jogos_resultados:  # Apenas adiciona se houver resultados
                    linha_completa.append(' '.join(ultimos_jogos_resultados))  # Adiciona como string
                else:
                    linha_completa.append('')  # Adiciona string vazia se não houver resultados

                # Verifica se a linha começa com "Classificação" e ignora
                if linha_completa[0] != "Classificação":  # Ignora a linha de título
                    tabela_completa.append(linha_completa)

# Imprimir a tabela
print(tabulate(tabela_completa, headers='firstrow', tablefmt='grid'))
